In [1]:
%%javascript

// ==================================================  //
// This part of the tutorial is for creating CHM maps //
// ==================================================  //

var study_polygon = /* color: #98ff00 */ee.Geometry.Polygon(
        [[[-83.5425714556907, 35.701602862979236],
          [-83.5422281329368, 35.690310616971246],
          [-83.53879490539774, 35.63828989061714],
          [-83.5037759844993, 35.615406781430224],
          [-83.48592430887022, 35.61938453875363],
          [-83.47459355041727, 35.627546807436595],
          [-83.47922808124717, 35.65117980654369],
          [-83.47738302453864, 35.682051678012314],
          [-83.4774257678571, 35.69326203209512],
          [-83.47751235961914, 35.71456632200802],
          [-83.47837061486136, 35.7182597023649],
          [-83.48043060302734, 35.72055934657099],
          [-83.53656387329102, 35.70494866628485]]]);

// Define the GRSM perim variable
var grsm_boundary = ee.FeatureCollection('projects/ee-stbr4432/assets/grsm_polygon');

// Define the fire perimeter variable
var fire_perimeter = ee.FeatureCollection('projects/ee-stbr4432/assets/chimney_tops_perim');
var fireBoundGeom = fire_perimeter.geometry();

// Apply the intersection method to the Polygon object.
var polygonIntersection = fireBoundGeom.intersection(grsm_boundary);
print('polygon intersection',polygonIntersection);

// Define study area.
var studyArea = study_polygon.intersection(polygonIntersection);

// Set center and zoom 
Map.setCenter(-83.5, 35.6, 11);


// ======= Read data, calculate and plot dCHM ======= //


// Read GRSM CHM data for 2016, 2017, 2021
var canopy_data = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30015-001');
var grsmSite = canopy_data.filter('NEON_SITE == "GRSM"');

var grsm_chm_2016 = grsmSite.filterDate('2016-01-01','2017-01-01').mosaic().clip(studyArea);
var grsm_chm_2017 = grsmSite.filterDate('2017-01-01','2018-01-01').mosaic().clip(studyArea);
var grsm_chm_2021 = grsmSite.filterDate('2021-01-01','2022-01-01').mosaic().clip(studyArea);

// Calculate CHMs of difference
var dCHM_2017_2016 = grsm_chm_2017.subtract(grsm_chm_2016);
var dCHM_2021_2017 = grsm_chm_2021.subtract(grsm_chm_2017);
var dCHM_2021_2016 = grsm_chm_2021.subtract(grsm_chm_2016);

// Set visualization parameters for CHM and dCHM images
var chm_vis = {min:0, max:40, palette:['lightgreen', 'green']};
var dchm_vis = {min:-30, max:30, palette:['red','white', 'green']};


// Add CHM layers to the map
Map.addLayer(grsm_chm_2016, chm_vis, 'CHM_2016');
Map.addLayer(grsm_chm_2017, chm_vis, 'CHM_2017');
Map.addLayer(grsm_chm_2021, chm_vis, 'CHM_2021');


// Add dCHM layers to the map
Map.addLayer(dCHM_2017_2016, dchm_vis, 'dCHM_2016/2017');
Map.addLayer(dCHM_2021_2017, dchm_vis, 'dCHM_2017/2021');
Map.addLayer(dCHM_2021_2016, dchm_vis, 'dCHM_2016/2021');


// Display the fire boundary
Map.addLayer(fire_perimeter.style({width: 2,
                                  color: "red",
                                  fillColor: "#00000000"}),{},"Fire Boundary", 1);
                                  


// [18:20] Bridget Hass (US)
// Define the histogram charts for each CHM difference image, print to the console.
var chm_hist_2016 = ui.Chart.image.histogram({image: grsm_chm_2016, region: studyArea, scale: 50, minBucketWidth: 2})
    .setOptions({title: 'CHM Histogram, 2016',
            hAxis: {title: 'CHM (m)',titleTextStyle: {italic: false, bold: true},},
            vAxis: {title: 'Count', titleTextStyle: {italic: false, bold: true}},});

print(chm_hist_2016);

var chm_hist_2017 = ui.Chart.image.histogram({image: grsm_chm_2017, region: studyArea, scale: 50, minBucketWidth: 2})
    .setOptions({title: 'CHM Histogram, 2017',
            hAxis: {title: 'CHM (m)',titleTextStyle: {italic: false, bold: true},},
            vAxis: {title: 'Count', titleTextStyle: {italic: false, bold: true}},});


print(chm_hist_2017);

var chm_hist_2021 = ui.Chart.image.histogram({image: grsm_chm_2021, region: studyArea, scale: 50, minBucketWidth: 2})
    .setOptions({title: 'CHM Histogram, 2021',
            hAxis: {title: 'CHM (m)',titleTextStyle: {italic: false, bold: true},},
            vAxis: {title: 'Count', titleTextStyle: {italic: false, bold: true}},});


print(chm_hist_2021);

// Mean Canopy Height by year
print('Mean Canopy Height 2016',grsm_chm_2016.reduceRegion({
      reducer: ee.Reducer.mean(),
      geometry: studyArea,
      scale : 250
      }));
      
print('Mean Canopy Height 2017',grsm_chm_2017.reduceRegion({
      reducer: ee.Reducer.mean(),
      geometry: studyArea,
      scale : 250
      }));
      
print('Mean Canopy Height 2021',grsm_chm_2021.reduceRegion({
      reducer: ee.Reducer.mean(),
      geometry: studyArea,
      scale : 250
      }));
      
// Uploading NEON All sites Boundaries 
// var AOP_Boundaries = ee.FeatureCollection('projects/ee-stbr4432/assets/AOP_flightboxesAllSites');
// Display the flight box boundary
// Map.addLayer(AOP_Boundaries.style({width: 3, color: "yellow", fillColor: "#00000000"}),{},"Flight Box", 1);

<IPython.core.display.Javascript object>